# Interactive Web application with Dash

Authors: Prof. med. Thomas Ganslandt <Thomas.Ganslandt@medma.uni-heidelberg.de> <br>
and Kim Hee <HeeEun.Kim@medma.uni-heidelberg.de> <br>
Heinrich-Lanz-Center for Digital Health (HLZ) of the Medical Faculty Mannheim <br>
Heidelberg University

This tutorial is prepared for TMF summer school on 03.07.2019

## Prerequisite: MIMIC-III files locally

You should place the following MIMIC-III data files in the data/ subfolder:

* D_LABITEMS.csv
* LABEVENTS.csv
* PRESCRIPTIONS.csv

### Dash
* https://dash.plot.ly/gallery <br>
* Dash is a Python framework for creating data-driven web applications <br>
* Dash apps are written on top of Flask, Plotly, and React
 * Flask is a Python web framework
 * Plotly is specifically a charting library built on top of D3.js
 * React is a JavaScript library for building user interfaces maintained by Facebook and a community 

## Case Study 2: Labitems Trend Visualization
[Trend analysis](https://en.wikipedia.org/wiki/Trend_analysis) is the widespread practice of collecting information and attempting to spot a pattern. This case study will illustrate a drug reaction of a sepsis patient. This case study tracks the biomarker and prescription history of patient 41976. It visualizes the relation between two key biomarkers of sepsis (White Blood Cells and Neutrophils) and 

* '41976' patient is choosen for this case study because this patient contains most and interesting records among other sepsis patients '10006', '10013', '10036', '10056', '40601'

### Import Python pakages (1/6)

In [1]:
# # Dash packages installation
# !conda install -c conda-forge dash-renderer -y
# !conda install -c conda-forge dash -y
# !conda install -c conda-forge dash-html-components -y
# !conda install -c conda-forge dash-core-components -y
# !conda install -c conda-forge plotly -y

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import flask
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import psycopg2
import getpass
pd.set_option('display.max_columns', 999)
import pandas.io.sql as psql

C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\2284574985.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\2284574985.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Data collection (2/6)
* Query `d_labitems` table (Dictionary table for mapping)
* Query `labevents` table (History of the labitem order)
* Join two tables
* Query `prescriptions` table (History of the prscription order)

In [3]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'

# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

In [8]:
query = \
"""
SELECT *
FROM D_LABITEMS limit 500;
"""

d_lab = pd.read_sql_query(query,con)

query_2 = \
"""
SELECT *
FROM LABEVENTS limit 500;
"""

lab = pd.read_sql_query(query_2,con)

C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\287491463.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d_lab = pd.read_sql_query(query,con)
C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\287491463.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lab = pd.read_sql_query(query_2,con)


In [9]:
lab.head()

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,None
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,None,None
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,None
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,None
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [10]:
d_lab.columns = map(str.lower, d_lab.columns)
d_lab.drop(columns = ['row_id'], inplace = True)

lab.columns = map(str.lower, lab.columns)
#lab = lab[lab['subject_id'] == 41976]
lab.drop(columns = ['row_id'], inplace = True)

lab = pd.merge(d_lab, lab, on = 'itemid', how = 'inner')
print(lab.columns)
lab[['subject_id', 'hadm_id', 'itemid', 'label', 'value']].head()

Index(['itemid', 'label', 'fluid', 'category', 'loinc_code', 'subject_id',
       'hadm_id', 'charttime', 'value', 'valuenum', 'valueuom', 'flag'],
      dtype='object')


,subject_id,hadm_id,itemid,label,value
0,3,NaN,51463,Bacteria,NONE
1,3,NaN,51464,Bilirubin,NEG
2,3,NaN,51464,Bilirubin,NEG
3,3,NaN,51466,Blood,SM
4,3,NaN,51466,Blood,NEG


In [12]:
query_3 = \
"""
SELECT *
FROM PRESCRIPTIONS limit 500;
"""

presc = pd.read_sql_query(query_3,con)

#presc = pd.read_csv("data/PRESCRIPTIONS.csv")
presc.columns = map(str.lower, presc.columns)
#presc = presc[presc['subject_id'] == 41976]
presc.drop(columns = ['row_id'], inplace = True)
print(presc.columns)
presc[['subject_id', 'hadm_id', 'icustay_id', 'drug']].head()

Index(['subject_id', 'hadm_id', 'icustay_id', 'startdate', 'enddate',
       'drug_type', 'drug', 'drug_name_poe', 'drug_name_generic',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'dose_val_rx',
       'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'route'],
      dtype='object')


C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\49138670.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  presc = pd.read_sql_query(query_3,con)


,subject_id,hadm_id,icustay_id,drug
0,6,107064,NaN,Tacrolimus
1,6,107064,NaN,Warfarin
2,6,107064,NaN,Heparin Sodium
3,6,107064,NaN,D5W
4,6,107064,NaN,Furosemide


###  Data preparation for labevents table (3/6)
* Convert data type to datetime and extract only year value 

In [13]:
lab['charttime'] = pd.to_datetime(lab['charttime'], errors = 'coerce')
lab.sort_values(by='charttime', inplace=True)
lab.set_index('charttime', inplace = True)
lab.head(1)

,itemid,label,fluid,category,loinc_code,subject_id,hadm_id,value,valuenum,valueuom,flag
charttime,,,,,,,,,,,
2101-10-12 16:07:00,50820,pH,Blood,Blood Gas,11558-4,3,NaN,7.39,7.39,units,None


###  Data preparation for prescriptions table (4/6)
* Filter conditions:
 * unit: 'mg'
 * antibiotics medicines: ('Vancomycin','Meropenem','Levofloxacin')
* Contruct a normalized dose column
* Convert data type to datetime and extract only year value 

In [14]:
presc['dose_val_rx'] = pd.to_numeric(presc['dose_val_rx'], errors = 'coerce')
presc = presc[presc['dose_unit_rx']=='mg']
presc = presc[presc['drug'].isin(['Vancomycin','Meropenem','Levofloxacin'])]

temp_df = pd.DataFrame()
for item in presc.drug.unique():
    temp = presc[presc['drug'].str.contains(item)]
    temp['norm_size'] = temp['dose_val_rx'] / temp['dose_val_rx'].max()
    temp_df = temp_df.append(temp)
presc = pd.merge(presc, temp_df, on=list(presc.columns))

presc['startdate'] = pd.to_datetime(presc['startdate'], errors = 'coerce')
presc.sort_values(by='startdate', inplace=True)
presc.set_index('startdate', inplace = True)
presc.head(1)

C:\Users\hassa\AppData\Local\Temp\ipykernel_18644\1064048487.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['norm_size'] = temp['dose_val_rx'] / temp['dose_val_rx'].max()


AttributeError: 'DataFrame' object has no attribute 'append'

###  Create a structure and presentation of your web with HTML and CSS (5/6)

In [ ]:
list_patient = ['41976']
list_biomarker = ['White Blood Cells', 'Neutrophils']
list_drug = ['Vancomycin','Meropenem','Levofloxacin']

# stylesheets = ['./resources/bWLwgP.css']
app = dash.Dash()

app.layout = html.Div([

    dcc.Dropdown(
        id = 'patient',
        value = '41976',
        multi = False,
        options = [{'label': i, 'value': i} for i in list_patient],
    ),
    dcc.Dropdown(
        id = 'biomarker',
        value = 'White Blood Cells',
        multi = False,
        options = [{'label': i, 'value': i} for i in list_biomarker],
    ),
    dcc.Dropdown(
        id = 'drug',
        value = ['Vancomycin'],
        multi = True,
        options = [{'label': i, 'value': i} for i in list_drug],
    ),
    dcc.Graph(id = 'graph'),
])

###  Define the reactive behavior with Python (6/6)

In [ ]:
@app.callback(Output('graph', 'figure'), 
              [Input('patient', 'value'),
               Input('biomarker', 'value'),
               Input('drug', 'value')])
def update_graph(patient, biomarker, drug):
    traces = []
    temp_l = lab[lab['subject_id'].astype(str) == patient]
    temp_p = presc[presc['subject_id'].astype(str) == patient]
    temp_min = 0
    
    item = biomarker
    temp = temp_l[temp_l['label'] == item]
    temp_min = float(temp.value.astype(float).min())
    trace = go.Scatter(
                x = temp.index,
                y = temp.value,
                name = item,
                mode = 'lines+markers',
            )
    traces.append(trace)
        
    for i, item in enumerate(drug):
        temp = temp_p[ temp_p['drug'] == item]
        trace = go.Scatter(
                    x = temp.index,
                    y = np.ones((1, len(temp)))[0] * temp_min - i - 1,
                    name = item,
                    mode = 'markers',
                    marker = {
                        'size': temp.norm_size * 10
                    }
                )
        traces.append(trace)
    
    layout = go.Layout(
        legend = {'x': 0.5, 'y': -0.1, 'orientation': 'h', 'xanchor': 'center'},
        margin = {'l': 300, 'b': 10, 't': 10, 'r': 300},
        hovermode = 'closest',
    )
    return {'data': traces, 'layout': layout}

In [ ]:
app.run_server(port = 8050)

## Takeaway
* Python is excellent for data science
 * Easy to analyze and visualize data
 * Quickly develop a data-driven web application

## External Resources (1/2)
* Introductory course of Python as a data analysis tool
 * Getting and Cleaning Data (Johns Hopkins University)
 * https://www.coursera.org/learn/data-cleaning#
* Introductory course of Data Science 
 * Foundations of Data Science — Spring 2016 (Berkeley University) 
 * https://data-8.appspot.com/sp16/

## External Resources (2/2)
* Docker
 * https://docs.docker.com/get-started/
 * deploy containerized instances
* Kaggle: the largest and most diverse data community in the world
 * Competitions and take an advantage of using “Kernels”
 * www.kaggle.com/competitions
 * Example kernels:
   * Can you improve lung cancer detection? (https://goo.gl/MV01o3)
   * Transforming How We Diagnose Heart Disease (https://goo.gl/b9Rta1)
   * Predict West Nile virus in mosquitos across the city of Chicago (https://goo.gl/VdVKtF)
   * and more
* Visualization examples
 * Matplotlib (https://matplotlib.org/gallery/index.html)
 * Plotly (https://plot.ly/python/)



# Question?
Authors: Prof. med. Thomas Ganslandt <Thomas.Ganslandt@medma.uni-heidelberg.de> <br>
and Kim Hee <HeeEun.Kim@medma.uni-heidelberg.de> <br>

Heinrich-Lanz-Center for Digital Health (HLZ) of the Medical Faculty Mannheim <br>
Heidelberg University

This is a part of a tutorial prepared for TMF summer school on 03.07.2019